In [22]:
%pylab nbagg
from tvb.simulator.lab import *
from tvb.analyzers
from tvb.datatypes.time_series import TimeSeriesSEEG

import os.path
from matplotlib import colors, cm
import time
import scipy.signal as sig
import scipy.io as sio
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
patient = 'id013_pg'
project_dir = '/Users/adam2392/Documents/pydata/metadata/' + patient

In [13]:
con = connectivity.Connectivity.from_file(os.path.join(project_dir, "connectivity.zip"))
con.speed = np.inf
con.cortical[:] = True     # To avoid adding analytical gain matrix for subcortical sources
# normalize
con.weights = con.weights/np.max(con.weights)
nb_regions = len(con.region_labels)

WARNING  File 'hemispheres' not found in ZIP.


In [19]:
x0ez=-1.6
x0pz=-2.4
x0num=-2.4

ezind = 0

coupl = coupling.Difference(a=1.)

#Initialise some Monitors with period in physical time
mon_tavg = monitors.TemporalAverage(period=1.)

print nb_regions

# Integrator
hiss = noise.Additive(nsig = numpy.array([0.001, 0.001, 0., 0.0001, 0.0001, 0.]))
heunint = integrators.HeunStochastic(dt=0.05, noise=hiss)


# Epileptor model
epileptors = models.Epileptor(Ks=-2, r=0.0002, tau = 10, tt = 0.07)
epileptors.x0 = -2.4*np.ones(nb_regions)
epileptors.x0[ezind] = -1.6

epileptors.state_variable_range['x1'] = r_[-0.5, 0.1]
epileptors.state_variable_range['z'] = r_[3.5,3.7]
epileptors.state_variable_range['y1'] = r_[-0.1,1]
epileptors.state_variable_range['x2'] = r_[-2.,0.]
epileptors.state_variable_range['y2'] = r_[0.,2.]
epileptors.state_variable_range['g'] = r_[-1.,1.]

# monitors
mon_tavg = monitors.TemporalAverage(period=1.0)
mon_SEEG = monitors.iEEG.from_file(sensors_fname=os.path.join(project_dir, "seeg.txt"),
                                   projection_fname=os.path.join(project_dir, "gain_inv-square.txt"),
                                   period=1.0,
                                   variables_of_interest=[0]
                                   )

num_contacts = mon_SEEG.sensors.labels.size

# run simulation
sim = simulator.Simulator(model=epileptors,
                          connectivity=con,
                          coupling=coupl,
                          conduction_speed=np.inf,                          
                          integrator=heunint,
                          monitors=[mon_tavg, mon_SEEG])

sim.configure()

(ttavg, tavg), (tseeg, seeg) = sim.run(simulation_length=10000)

# Normalize the time series to have nice plots
tavgn = tavg/(np.max(tavg, 0) - np.min(tavg, 0))
seegn = seeg/(np.max(seeg, 0) - np.min(seeg, 0))
seegn = seegn - np.mean(seegn, 0)

84


In [20]:
print(ttavg.shape)
print(tavg.shape)
print(seeg.shape)

(10000,)
(10000, 2, 84, 1)
(10000, 1, 59, 1)


# Setup Data to Be Analyzed By Fragility

0. Add randomized line noise if not added with somewhat wide bandwidths (optional)
1. Filter line noise with narrow bands
2. Compute the MVAR models
3. Compute the fragility models

In [21]:
b, a = sig.butter(2, 0.1, btype='highpass', output='ba')
seegf = np.zeros(seegn.shape)
for i in range(num_contacts):
    seegf[:, 0, i, 0] = sig.filtfilt(b, a, seeg[:, 0, i, 0])

In [23]:
# initialize models
mvarcomp = fragility.mvarwindowed()
fragcomp = fragility.fragilitymodel()

AttributeError: 'module' object has no attribute 'mvarwindowed'

In [ ]:
tsr = TimeSeriesRegion(connectivity=sim.connectivity,
                       data=tavg_samples,
                       sample_period=sim.monitors[0].period)
tsr.configure()

In [ ]:
corrcoeff_analyser = corr_coeff.CorrelationCoefficient(time_series=tsr)
corrcoeff_data = corrcoeff_analyser.evaluate()
corrcoeff_data.configure()
FC = corrcoeff_data.array_data[..., 0, 0]

In [ ]:
plot_tri_matrix(FC,
                cmap=pyplot.cm.RdYlBu_r, 
                node_labels= sim.connectivity.region_labels,
                size=[10., 10.],
                color_anchor=(-1.0, 1.0));